In [2]:
#import json # used as a lightweight data interchange format.

import pandas as pd # 판다스 import
import torch
from matplotlib import pyplot as plt # matplotlib에서 pypplot 다운로드
 
from tqdm import tqdm # tqdm import

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
train_path = '../data/train.csv'
dev_path = '../data/dev.csv'

In [18]:
# train_data와 dev_data 전체를 concat하여 각 label 값의 범위에 대한 데이터 개수 구하는 함수
def counts_data_range(train_path, dev_path):
    train_df = pd.read_csv(train_path) # train data 읽어오기
    dev_df = pd.read_csv(dev_path) # dev data 읽어오기
    data_df = pd.concat([train_df, dev_df], axis=0) # train data와 dev data에 대해 axis 0 기준으로 concat

    bins = pd.cut(data_df['label'], bins=[0, 1, 2, 3, 4, 5], include_lowest=True) # 모든 row를 읽어와 그 row의 'label' column에 대한 값이 해당되는 구간을 맵핑하여 할당
    counts = bins.value_counts(sort=False) # 각 구간별로 개수를 세기
    data_counts = [counts[i] for i in range(1, len(counts) + 1)] # 각 구간별 개수를 추출
    return data_counts

print(counts_data_range(train_path, dev_path))

[4163, 1372, 1294, 2058, 987]


In [31]:
train_df = pd.read_csv(train_path) # train data 읽어오기
dev_df = pd.read_csv(dev_path) # dev data 읽어오기
data_df = pd.concat([train_df, dev_df], axis=0)

In [35]:
#label_counts = data_df['label'].value_counts().sort_index() # 각 label값에 해당하는 데이터 개수 계산 (label값 기준으로 오름차순 정렬)
label_counts = data_df['label'].value_counts() # 각 label값에 해당하는 데이터 개수 계산 (데이터 개수를 기준으로 내림차순 정렬)
print(label_counts)

0.0    2140
4.0     538
0.4     498
3.8     474
4.2     451
3.6     419
0.6     392
0.8     380
1.0     364
1.2     359
0.2     352
3.0     305
3.4     301
3.2     292
2.8     289
1.4     271
1.8     268
2.0     258
2.6     257
4.4     228
2.2     213
2.4     208
1.6     190
5.0     113
4.6      96
4.8      81
0.5      37
3.5      34
1.5      26
2.5      22
4.5      18
Name: label, dtype: int64


In [36]:
label_ratios = label_counts / label_counts.sum() # 전체 데이터에 대한 각 label값에 해당하는 데이터 수 비율 계산
print(label_ratios)

0.0    0.216731
4.0    0.054487
0.4    0.050435
3.8    0.048005
4.2    0.045676
3.6    0.042435
0.6    0.039700
0.8    0.038485
1.0    0.036864
1.2    0.036358
0.2    0.035649
3.0    0.030889
3.4    0.030484
3.2    0.029573
2.8    0.029269
1.4    0.027446
1.8    0.027142
2.0    0.026129
2.6    0.026028
4.4    0.023091
2.2    0.021572
2.4    0.021065
1.6    0.019242
5.0    0.011444
4.6    0.009723
4.8    0.008203
0.5    0.003747
3.5    0.003443
1.5    0.002633
2.5    0.002228
4.5    0.001823
Name: label, dtype: float64


# Weighted MSE LOSS

In [13]:
# 작성한 함수 테스트를 위한 sample 데이터
y_true = torch.tensor([2.4, 1.8, 3.1, 4.0, 2.9])
y_pred = torch.tensor([2.5, 1.6, 2.9, 4.1, 2.8\])
#y_true = torch.tensor([2.4])
#y_pred = torch.tensor([2.5])

# 데이터 개수 {(-0.001, 1.0] : 4163, (1.0, 2.0] : 1372, (2.0, 3.0] : 1294, (3.0, 4.0] : 2058, (4.0, 5.0] : 987}
data_counts = counts_data_range(train_path, dev_path)

'''
(-0.001, 1.0]    4163
(3.0, 4.0]       2058
(1.0, 2.0]       1372
(2.0, 3.0]       1294
(4.0, 5.0]        987
'''

# 각 구간별 가중치를 계산하는 함수
def calculate_weights(bin_counts):
    total_samples = sum(bin_counts)  # 전체 데이터 개수
    bin_ratios = [count / total_samples for count in bin_counts]  # 각 구간별 비율
    bin_weights = [1 / (ratio * len(bin_counts)) for ratio in bin_ratios]  # 각 구간별 가중치
    return torch.tensor(bin_weights)

def weighted_MSE(data_counts, y_true, y_pred):
    
    # weights 계산
    weights = calculate_weights(data_counts) # 결과 예) tensor([0.4744, 1.4394, 1.5261, 0.9596, 2.0008])
    print(f'weights : {weights}')

    # 각 실제값(label)이 속한 구간의 인덱스를 계산
    # bins = torch.tensor([0, 1, 2, 3, 4]) # (-0.001, 1.0], (1.0, 2.0], (2.0, 3.0], (3.0, 4.0], (4.0, 5.0]
    # bin_indices = torch.bucketize(y_true, bins)
    bin_indices = torch.bucketize(y_true, torch.tensor([0, 1, 2, 3, 4]))
    print(f'bin_indices : {bin_indices}')
    
    # 각 label에 대한 weight 설정
    bin_weights = weights[bin_indices - 1]
    print(f'bin_weights : {bin_weights}')

    # Weighted MSE 계산
    mse = torch.mean(bin_weights * (y_true - y_pred) ** 2) 
    
    return mse

print("Weighted MSE:", weighted_MSE(data_counts, y_true, y_pred).item())

weights : tensor([0.4744, 1.4394, 1.5261, 0.9596, 2.0008])
bin_indices : tensor([3, 2, 4, 4])
bin_weights : tensor([1.5261, 1.4394, 0.9596, 0.9596])
Weighted MSE: 0.030203502625226974


In [19]:
y_true = torch.tensor([-0.00001, 1.8, 3.1, 4.0, 7.4])
bins = torch.tensor([0, 1, 2, 3, 4])
bin_indices = torch.bucketize(y_true, bins) # (0, 1], (1, 2], (2, 3], (3, 4], (4, 5]
print(bin_indices)

tensor([0, 2, 4, 4, 5])


# pred_data에 대한 학습된 모델의 예측 결과 분석 (Test 결과 분석)

테스트에 사용한 dev 데이터 분석

In [20]:
# csv 파일을 읽어들이기
df = pd.read_csv('./data/dev.csv')

# 'target' column의 범위를 나누어서 해당 범위에 속하는 row의 개수를 카운트
bins = [-0.001, 1.0, 2.0, 3.0, 4.0, 5.0]
labels = ['(-0.001, 1.0]', '(1.0, 2.0]', '(2.0, 3.0]', '(3.0, 4.0]', '(4.0, 5.0]']
df['target_range'] = pd.cut(df['label'], bins=bins, labels=labels)
count = df['target_range'].value_counts()

print(count)

(-0.001, 1.0]    110
(1.0, 2.0]       110
(2.0, 3.0]       110
(3.0, 4.0]       110
(4.0, 5.0]       110
Name: target_range, dtype: int64


weighted MSE Loss를 사용하여 학습된 모델의 dev 데이터에 대한 예측 결과 분석

In [5]:
# csv 파일을 읽어들이기
df = pd.read_csv('./my_log/2023-04-21-15:01:22/wrong.csv')

# 'pred'와 'target' column의 차이가 1.0 이상인 데이터를 추출
diff = abs(df['pred'] - df['target'])
df = df[diff >= 1.0]

# 'target' column의 범위를 나누어서 해당 범위에 속하는 row의 개수를 카운트
bins = [-0.001, 1.0, 2.0, 3.0, 4.0, 5.0]
labels = ['(-0.001, 1.0]', '(1.0, 2.0]', '(2.0, 3.0]', '(3.0, 4.0]', '(4.0, 5.0]']
df['target_range'] = pd.cut(df['target'], bins=bins, labels=labels)
count = df['target_range'].value_counts(sort=False)

print(count)

(-0.001, 1.0]    17
(1.0, 2.0]       20
(2.0, 3.0]       24
(3.0, 4.0]       16
(4.0, 5.0]       13
Name: target_range, dtype: int64


MSE Loss를 사용하여 학습된 모델의 dev 데이터에 대한 예측 결과 분석

In [6]:
# csv 파일을 읽어들이기
df = pd.read_csv('./my_log/2023-04-20-00:07:10/wrong.csv')

# 'pred'와 'target' column의 차이가 1.0 이상인 데이터를 추출
diff = abs(df['pred'] - df['target'])
df = df[diff >= 1.0]

# 'target' column의 범위를 나누어서 해당 범위에 속하는 row의 개수를 카운트
bins = [-0.001, 1.0, 2.0, 3.0, 4.0, 5.0]
labels = ['(-0.001, 1.0]', '(1.0, 2.0]', '(2.0, 3.0]', '(3.0, 4.0]', '(4.0, 5.0]']
df['target_range'] = pd.cut(df['target'], bins=bins, labels=labels)
count = df['target_range'].value_counts(sort=False)

print(count)

(-0.001, 1.0]     3
(1.0, 2.0]       16
(2.0, 3.0]       25
(3.0, 4.0]       10
(4.0, 5.0]        8
Name: target_range, dtype: int64
